In [55]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, LineString
import os
import math
import json
import gzip
import time
import calendar
from datetime import datetime
import numpy as np
import pandas as pd
from datetime import date
from multiprocessing import Pool
import warnings
warnings.filterwarnings('ignore')
SETS = './sets/' # папка с файлами с наборами
LONG = 0.0159 # долгота расстояние 1 км от точки на восток или на запад
LAT = 0.009 # широта расстояние 1 км от точки на север или на юг

In [65]:
def default_serializer(obj): # для правильного преобразования даты в ISO формат
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')
    

def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')
        

def distance(lon1, lat1, lon2, lat2):
    """
    Функция для вычисления расстояния между двумя точками на плоскости по их GPS координатам
    """
    R = 6371  # Радиус Земли в километрах
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c


# def index(exams, ecology):
#     counter = 0
#     for i, row in ecology.iterrows():
#         if exams['datesSet'].intersection(row['datesSet']):
#             a_list = exams['geoList']
#             for a in range(0, len(a_list), 2):
#                 a_longlat = a_list[a:a+2]
#                 if len(a_longlat) >= 2: # если в списке больше двух точек
#                     a_long = a_longlat[0]
#                     a_lat = a_longlat[1]
#                 b_list = row['geoList']
#                 for b in range(0, len(b_list), 2):
#                     b_longlat = b_list[b:b+2]
#                     if len(b_longlat) >= 2:
#                         b_long = b_longlat[0]
#                         b_lat = b_longlat[1]
#                     km = distance(a_long, a_lat, b_long, b_lat)
#                     #print(f'расстояние {km} между [ {a_long}, {a_lat} ] и [ {b_long}, {b_lat} ]')
#                     if km < 1:
#                         counter += 1


def calculate_distances(exams_row, ecology_rows):
    counter = 0
    a_list = exams_row['geoList']
    
    for b_row in ecology_rows.itertuples(index=False):
        if exams_row['datesSet'].intersection(b_row.datesSet):
            b_list = b_row.geoList
            for a in range(0, len(a_list), 2):
                if len(a_list) < 2:
                    continue
                a_long = a_list[a]
                a_lat = a_list[a+1]
                for b in range(0, len(b_list), 2):
                    if len(b_list) < 2:
                        continue
                    b_long = b_list[b]
                    b_lat = b_list[b + 1]
                    km = distance(a_long, a_lat, b_long, b_lat)
                    if km < 1:
                        counter += 1
    return counter

def index(exams, ecology):
    with Pool() as pool:
        results = pool.starmap(calculate_distances, [(row, ecology) for row in exams.itertuples(index=False)])
    
    total_count = sum(results)
    return total_count
    
            
def geo_list(geodata):
    if type(geodata) == str:
        try: 
            geodata = eval(geodata)
        except Exeption as e:
            print(geodata)
    geolist = []
    try:
        geolist = list(map(float, str(geodata['coordinates']).replace(',[]', '').replace('[', '').replace(']', '').split(',')))
    except Exception as e:
        print(e, geodata)
    return geolist


def dates_set(start, end):
    #start_date = datetime.strptime(start, '%d.%m.%Y')
    #end_date = datetime.strptime(end, '%d.%m.%Y')
    exams_range = pd.date_range(start=start, end=end)
    return set(exams_range)


def sets_intersects(exams_set, start, end):
    """
    Для уменьшения объема датасета ecology, проводим проверку, входят ли даты событий в наборы дат экзаменов
    """
    #date_start = datetime.strptime(start, '%d.%m.%Y')
    #date_end = datetime.strptime(end, '%d.%m.%Y')
    date_range = pd.date_range(start=start, end=end)
    date_set = set(date_range)
    if date_set.intersection(exams_set):
        return True
    else:
        return False

In [6]:
pd.set_option('display.max_colwidth', None) # для отображения полного текста в ячейках
pd.set_option('display.max_columns', None) 
ecology_lite = load_dataset('ecology_lite')
#ecology_lite['start'] = pd.to_datetime(ecology_lite['start'], format='%Y-%m-%d')
#ecology_lite['end'] = pd.to_datetime(ecology_lite['end'], format='%Y-%m-%d')
exams_lite = load_dataset('exams_lite')
#exams_lite['start'] = pd.to_datetime(exams_lite['start'], format='%Y-%m-%d')
#exams_lite['end'] = pd.to_datetime(exams_lite['end'], format='%Y-%m-%d')

In [7]:
#exams_lite['start'] = pd.to_datetime(exams_lite['start'], format='%d.%m.%Y')
#exams_lite['end'] = pd.to_datetime(exams_lite['end'], format='%d.%m.%Y')
exams_lite.head()

,global_id,start,end,stupid,geoData
0,797946035,2017-03-01T00:00:00,2017-09-30T00:00:00,3.800000,"{'type': 'Point', 'coordinates': [37.723652, 55.814547]}"
1,797949150,2017-03-01T00:00:00,2017-09-30T00:00:00,3.357143,"{'coordinates': [[[[37.710083127, 55.825085846], [37.709884644, 55.825088859], [37.70991683, 55.825557425], [37.710107267, 55.825552906], [37.710083127, 55.825085846]]], [[[37.710971466, 55.824666254], [37.710390321, 55.824675741], [37.710401249, 55.824826407], [37.710980407, 55.824816363], [37.710971466, 55.824666254]]], [[[37.710757032, 55.82506651], [37.710557953, 55.825070528], [37.710585371, 55.825566047], [37.710780874, 55.825561359], [37.710757032, 55.82506651]]], [[[37.707049481, 55.825154524], [37.707242202, 55.825149502], [37.707241209, 55.825030085], [37.70704054, 55.82503176], [37.707049481, 55.825154524]]], [[[37.710930347, 55.825726055], [37.711380243, 55.825718823], [37.711373091, 55.825557312], [37.711369514, 55.825539635], [37.710924625, 55.825550081], [37.710924625, 55.825570973], [37.710930347, 55.825726055]]], [[[37.583026114, 55.832647996], [37.582666499, 55.832611174], [37.582629743, 55.832706019], [37.582630736, 55.832758463], [37.58344732, 55.83284717], [37.583486063, 55.832741167], [37.583481096, 55.83270044], [37.583129428, 55.832659155], [37.583156251, 55.832582163], [37.584034426, 55.832677566], [37.584082109, 55.83255092], [37.584084096, 55.832518561], [37.583018166, 55.832398051], [37.582968496, 55.832531393], [37.582976443, 55.832562078], [37.583054923, 55.832573236], [37.583026114, 55.832647996]]], [[[37.709097862, 55.825586152], [37.70911789, 55.825928458], [37.70928669, 55.825930065], [37.709272385, 55.825774179], [37.70981884, 55.825766144], [37.709795952, 55.825566867], [37.709097862, 55.825586152]]]], 'type': 'MultiPolygon'}"
2,797949275,2017-03-01T00:00:00,2017-09-30T00:00:00,1.100000,"{'coordinates': [[[[37.807984129, 55.726758653], [37.808307186, 55.726841896], [37.808526531, 55.726567328], [37.808196321, 55.726480727], [37.807984129, 55.726735157], [37.807984129, 55.726758653]]], [[[37.808976173, 55.726681066], [37.808727265, 55.726995241], [37.808704376, 55.727030686], [37.808858872, 55.727067742], [37.808921814, 55.726979129], [37.808944702, 55.72695174], [37.80899334, 55.726892127], [37.809216499, 55.726948517], [37.80932808, 55.726974296], [37.809522629, 55.72702263], [37.809602738, 55.726898572], [37.809597015, 55.72684057], [37.808976173, 55.726681066]]], [[[37.809706703, 55.726866063], [37.809706703, 55.726887545], [37.809673324, 55.72692044], [37.809673324, 55.726957362], [37.809852138, 55.727005025], [37.810194269, 55.726581425], [37.810197845, 55.726525035], [37.810020223, 55.7264767], [37.809706703, 55.726866063]]], [[[37.808979526, 55.726210184], [37.808632627, 55.726643858], [37.808636203, 55.726692864], [37.808812633, 55.726735828], [37.809155956, 55.726312897], [37.809153572, 55.726253149], [37.808979526, 55.726210184]]]], 'type': 'MultiPolygon'}"
3,797949425,2017-03-01T00:00:00,2017-09-30T00:00:00,6.600000,"{'type': 'Point', 'coordinates': [37.786873, 55.762276]}"
4,797949570,2017-03-01T00:00:00,2017-09-30T00:00:00,5.000000,"{'type': 'Point', 'coordinates': [37.775102, 55.806463]}"


In [8]:
%%time
exams_lite['datesSet'] = exams_lite.apply(lambda x: dates_set(x['start'], x['end']), axis=1)

CPU times: user 3.38 s, sys: 72.3 ms, total: 3.45 s
Wall time: 3.46 s


In [9]:
exams_lite.head()

,global_id,start,end,stupid,geoData,datesSet
0,797946035,2017-03-01T00:00:00,2017-09-30T00:00:00,3.800000,"{'type': 'Point', 'coordinates': [37.723652, 55.814547]}","{2017-09-27 00:00:00, 2017-05-20 00:00:00, 2017-06-19 00:00:00, 2017-03-24 00:00:00, 2017-05-08 00:00:00, 2017-09-10 00:00:00, 2017-08-21 00:00:00, 2017-08-20 00:00:00, 2017-03-14 00:00:00, 2017-07-13 00:00:00, 2017-07-22 00:00:00, 2017-09-30 00:00:00, 2017-08-09 00:00:00, 2017-04-09 00:00:00, 2017-08-17 00:00:00, 2017-04-18 00:00:00, 2017-03-20 00:00:00, 2017-04-14 00:00:00, 2017-05-06 00:00:00, 2017-06-11 00:00:00, 2017-04-30 00:00:00, 2017-05-29 00:00:00, 2017-09-29 00:00:00, 2017-09-14 00:00:00, 2017-03-02 00:00:00, 2017-07-05 00:00:00, 2017-09-02 00:00:00, 2017-05-16 00:00:00, 2017-04-06 00:00:00, 2017-06-28 00:00:00, 2017-09-09 00:00:00, 2017-08-03 00:00:00, 2017-06-24 00:00:00, 2017-07-23 00:00:00, 2017-08-05 00:00:00, 2017-08-04 00:00:00, 2017-08-12 00:00:00, 2017-09-21 00:00:00, 2017-07-21 00:00:00, 2017-09-26 00:00:00, 2017-05-07 00:00:00, 2017-06-04 00:00:00, 2017-07-31 00:00:00, 2017-03-05 00:00:00, 2017-07-27 00:00:00, 2017-09-22 00:00:00, 2017-03-06 00:00:00, 2017-04-13 00:00:00, 2017-03-25 00:00:00, 2017-09-28 00:00:00, 2017-03-17 00:00:00, 2017-04-19 00:00:00, 2017-09-01 00:00:00, 2017-08-26 00:00:00, 2017-04-29 00:00:00, 2017-06-15 00:00:00, 2017-07-12 00:00:00, 2017-09-16 00:00:00, 2017-04-20 00:00:00, 2017-04-04 00:00:00, 2017-06-22 00:00:00, 2017-08-10 00:00:00, 2017-07-15 00:00:00, 2017-09-25 00:00:00, 2017-06-06 00:00:00, 2017-06-05 00:00:00, 2017-06-08 00:00:00, 2017-08-16 00:00:00, 2017-05-13 00:00:00, 2017-09-04 00:00:00, 2017-07-03 00:00:00, 2017-07-24 00:00:00, 2017-08-24 00:00:00, 2017-04-01 00:00:00, 2017-05-14 00:00:00, 2017-09-12 00:00:00, 2017-06-30 00:00:00, 2017-03-28 00:00:00, 2017-07-04 00:00:00, 2017-07-11 00:00:00, 2017-03-29 00:00:00, 2017-05-27 00:00:00, 2017-07-08 00:00:00, 2017-03-03 00:00:00, 2017-06-16 00:00:00, 2017-07-17 00:00:00, 2017-05-04 00:00:00, 2017-05-22 00:00:00, 2017-09-18 00:00:00, 2017-04-02 00:00:00, 2017-09-23 00:00:00, 2017-03-30 00:00:00, 2017-08-13 00:00:00, 2017-05-05 00:00:00, 2017-06-10 00:00:00, 2017-06-25 00:00:00, 2017-05-12 00:00:00, 2017-07-06 00:00:00, 2017-06-03 00:00:00, 2017-04-26 00:00:00, ...}"
1,797949150,2017-03-01T00:00:00,2017-09-30T00:00:00,3.357143,"{'coordinates': [[[[37.710083127, 55.825085846], [37.709884644, 55.825088859], [37.70991683, 55.825557425], [37.710107267, 55.825552906], [37.710083127, 55.825085846]]], [[[37.710971466, 55.824666254], [37.710390321, 55.824675741], [37.710401249, 55.824826407], [37.710980407, 55.824816363], [37.710971466, 55.824666254]]], [[[37.710757032, 55.82506651], [37.710557953, 55.825070528], [37.710585371, 55.825566047], [37.710780874, 55.825561359], [37.710757032, 55.82506651]]], [[[37.707049481, 55.825154524], [37.707242202, 55.825149502], [37.707241209, 55.825030085], [37.70704054, 55.82503176], [37.707049481, 55.825154524]]], [[[37.710930347, 55.825726055], [37.711380243, 55.825718823], [37.711373091, 55.825557312], [37.711369514, 55.825539635], [37.710924625, 55.825550081], [37.710924625, 55.825570973], [37.710930347, 55.825726055]]], [[[37.583026114, 55.832647996], [37.582666499, 55.832611174], [37.582629743, 55.832706019], [37.582630736, 55.832758463], [37.58344732, 55.83284717], [37.583486063, 55.832741167], [37.583481096, 55.83270044], [37.583129428, 55.832659155], [37.583156251, 55.832582163], [37.584034426, 55.832677566], [37.584082109, 55.83255092], [37.584084096, 55.832518561], [37.583018166, 55.832398051], [37.582968496, 55.832531393], [37.582976443, 55.832562078], [37.583054923, 55.832573236], [37.583026114, 55.832647996]]], [[[37.709097862, 55.825586152], [37.70911789, 55.825928458], [37.70928669, 55.825930065], [37.709272385, 55.825774179], [37.70981884, 55.825766144], [37.709795952, 55.825566867], [37.709097862, 55.825586152]]]], 'type': 'MultiPolygon'}","{2017-09-27 00:00:00, 2017-05-20 00:00:00, 2017-06-19 00:00:00, 2017-03-24 

In [10]:
exams_lite['geoList'] = exams_lite.apply(lambda x: geo_list(x['geoData']), axis=1)

In [11]:
%%time
ecology_lite['datesSet'] = ecology_lite.apply(lambda x: dates_set(x['start'], x['end']), axis=1)

CPU times: user 19.3 s, sys: 735 ms, total: 20.1 s
Wall time: 20.1 s


In [12]:
ecology_lite['geoList'] = ecology_lite.apply(lambda x: geo_list(x['stroi']), axis=1)

In [13]:
ecology_lite.head()

,start,end,stroi,datesSet,geoList
0,2017-01-01T00:00:00,2022-12-31T00:00:00,"{'type': 'MultyPolygon', 'coordinates': '[[[37.677473549188,55.641930681152],[37.677826849354,55.642236011443],[37.677883129013,55.642284613055],[37.677863785761,55.642291736661],[37.677918471358,55.642338993246],[37.677937814623,55.642331869631],[37.677964758224,55.642355004473],[37.678296602109,55.642232101742],[37.678450869815,55.642174933309],[37.67852237499,55.642148423006],[37.678622577603,55.642111362267],[37.678549557193,55.642048413931],[37.67854051992,55.642051750321],[37.678520384465,55.642059234455],[37.678510554344,55.642062841448],[37.67848712879,55.642045090858],[37.678354862842,55.641944952508],[37.678172399845,55.641806622933],[37.678148178521,55.641788424323],[37.678052883981,55.641716166467],[37.677723421819,55.641838166929],[37.677679504205,55.641854487467],[37.677550762511,55.641902097362],[37.677515406053,55.641915172027],[37.677473549188,55.641930681152]],[[37.677826849354,55.642236011443],[37.677807506116,55.642243135041],[37.677863785761,55.642291736661],[37.677918471358,55.642338993246],[37.677937814623,55.642331869631],[37.677964758224,55.642355004473],[37.678296602109,55.642232101742],[37.678450869815,55.642174933309],[37.678463146565,55.642185604197],[37.678534651751,55.642159093888],[37.67852237499,55.642148423006],[37.678622577603,55.642111362267],[37.678549557193,55.642048413931],[37.67854051992,55.642051750321],[37.678537969206,55.642049598319],[37.678517833751,55.642057082453],[37.678520384465,55.642059234455],[37.678510554344,55.642062841448],[37.67848712879,55.642045090858],[37.678354862842,55.641944952508],[37.678172399845,55.641806622933],[37.678189203852,55.641799952036],[37.678164663286,55.641781394608],[37.678148178521,55.641788424323],[37.678052883981,55.641716166467],[37.677723421819,55.641838166929],[37.677707638229,55.641824536939],[37.677663561409,55.64184076788],[37.677679504205,55.641854487467],[37.677550762511,55.641902097362],[37.677534978552,55.64188837753],[37.677499622504,55.64190154201],[37.677515406053,55.641915172027],[37.677473549188,55.641930681152],[37.677826849354,55.642236011443]],[[37.677473549188,55.641930681152],[37.677826849354,55.642236011443],[37.677807506116,55.642243135041],[37.677863785761,55.642291736661],[37.677918471358,55.642338993246],[37.677937814623,55.642331869631],[37.677964758224,55.642355004473],[37.678296602109,55.642232101742],[37.678308878835,55.642242772646],[37.678463146565,55.642185604197],[37.678450869815,55.642174933309],[37.67852237499,55.642148423006],[37.678622577603,55.642111362267],[37.678549557193,55.642048413931],[37.678528192534,55.642029851993],[37.67848712879,55.642045090858],[37.678354862842,55.641944952508],[37.67837182326,55.641937742439],[37.678189203852,55.641799952036],[37.678172399845,55.641806622933],[37.678148178521,55.641788424323],[37.678052883981,55.641716166467],[37.677723421819,55.641838166929],[37.677679504205,55.641854487467],[37.677550762511,55.641902097362],[37.677515406053,55.641915172027],[37.677473549188,55.641930681152]]]'}","{2019-09-17 00:00:00, 2020-01-20 00:00:00, 2018-07-26 00:00:00, 2018-01-11 00:00:00, 2017-12-20 00:00:00, 2017-09-10 00:00:00, 2020-12-01 00:00:00, 2019-05-08 00:00:00, 2022-08-09 00:00:00, 2018-01-31 00:00:00, 2020-07-28 00:00:00, 2021-09-26 00:00:00, 2021-04-11 00:00:00, 2017-04-30 00:00:00, 2021-02-11 00:00:00, 2019-07-23 00:00:00, 2018-04-03 00:00:00, 2020-11-24 00:00:00, 2018-03-04 00:00:00, 2020-05-30 00:00:00, 2022-07-15 00:00:00, 2021-11-23 00:00:00, 2021-09-21 00:00:00, 2018-10-17 00:00:00, 2017-09-02 00:00:00, 2020-11-14 00:00:00, 2021-10-31 00:00:00, 2017-06-28 00:00:00, 2020-01-23 00:00:00, 2019-04-11 00:00:00, 2019-08-24 00:00:00, 2022-12-30 00:00:00, 2018-04-28 00:00:00, 2022-05-08 00:00:00, 2017-03-25 00:00:00, 2017-12-17 00:00:00, 2021-02-23 00:00:00, 2022-08-04 00:00:00, 2021-05-24 00:00:00, 2018-10-29 00:00:00, 2018-01-27 00:00:00, 2022-08-03 00:00:00, 2019-04-10 00:00:00, 2018-03-22 00:00:00,

In [66]:
# exams_lite.apply(lambda x: index(x, ecology_lite), axis=1)
result = index(exams_lite, ecology_lite)

PicklingError: Can't pickle <class 'pandas.core.frame.Pandas'>: attribute lookup Pandas on pandas.core.frame failed

In [59]:
%%time
index(exams_lite.iloc[1], ecology_lite)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/lab/env/lib64/python3.9/site-packages/IPython/core/magics/execution.py", line 1332, in time
    out = eval(code, glob, local_ns)
  File "<timed eval>", line 1, in <module>
  File "/tmp/ipykernel_1750/1318306702.py", line 83, in index
    results = pool.map(lambda row: calculate_distances(row, ecology), exams.itertuples(index=False))
  File "/opt/lab/env/lib64/python3.9/site-packages/pandas/core/generic.py", line 6296, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'Series' object has no attribute 'itertuples'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/lab/env/lib64/python3.9/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/opt/lab/env/lib64/python3.9/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.s